In [17]:
from pydantic import BaseModel, ConfigDict, Field
import numpy as np
from itertools import product
from random import choices


In [25]:
INIT_VEL = np.zeros(2, dtype=np.float32)

## Boids implementation

In [ ]:
class Boid(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)
    position: np.ndarray
    velocity: np.ndarray

def generate_swarm(n: int, x_max: int, y_max: int) -> list[Boid]:
    xs, ys = np.meshgrid(np.arange(x_max), np.arange(y_max))
    pairs = np.column_stack((xs.ravel(), ys.ravel()))
    idxs = np.random.choice(x_max * y_max, size=n, replace=False)
    return [Boid(position=pairs[i], velocity=INIT_VEL) for i in idxs]


def in_radius(x, y, r):
    return np.sqrt(np.sum((x-y)**2)) <= r
    

def compute_separation_vector(target, swarm, r):
    avg_vel
    for boid in swarm:    


class Rules(BaseModel):
    rs: float =  Field(description="radii_separation")
    ws: float =  Field(description="separation weight")
    ra: float =  Field(description="radii_alignment")
    wa: float =  Field(description="alignment weight")
    rc: float =  Field(description="radii_cohesion")
    

class World:
    def __init__(self, size: int) -> None:
        self._size = size
        self._boids: list[Boid] = []
    
    def set_swarm(self, swarm: list[Boid]) -> None:
        self._boids = swarm

np.True_

In [ ]:
x = BoidsSwarm(10)
x.generate_population(100, 1.0)

ValidationError: 1 validation error for Boid
velocity
  Input should be an instance of float64 [type=is_instance_of, input_value=1.0, input_type=float]
    For further information visit https://errors.pydantic.dev/2.11/v/is_instance_of